In [ ]:
from omegaconf import OmegaConf
import os
import panel as pn
import numpy as np
import pyvista as pv

from torch_points3d.datasets.segmentation.ept_dataset import EptDataset
pn.extension('vtk')
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'


In [ ]:
DIR = os.path.dirname(os.getcwd())
dataset_options = OmegaConf.load(os.path.join(DIR,'../conf/data/segmentation/ept/ground/ground.yaml'))

In [ ]:
dataset_options.data.dataroot = "/mnt/c/data/potreeconvert"
dataset = EptDataset(dataset_options.data)
print(dataset)

In [ ]:
x = dataset.train_dataset[0]

In [ ]:
x.y.max()

In [ ]:
x.pos.min(axis=0)

In [ ]:
x.pos.max(axis=0)

In [ ]:
x.pos.mean(axis=0)

In [ ]:
dataset.train_dataset.files[0]

In [ ]:
(x.pos.max() - x.pos.min()) / 2

In [ ]:
len(x.pos)

In [ ]:
from joblib import Parallel, delayed
import json
from tqdm import tqdm
import time
import laspy

in_dir = "/mnt/c/data/potreeconvert/ahn"

heirarchyPath = os.path.join(in_dir, "ept-hierarchy", "0-0-0-0.json")

print("[powerlineProcess] reading ept file %s: " % (heirarchyPath))
start_time = time.time()

with open(heirarchyPath) as f:
    hierarchy = json.load(f)
filelist = [os.path.join(in_dir, "ept-data", x + ".laz") for x in hierarchy.keys()]

def _read_las(file):
    las = laspy.read(file)            
    classes = np.copy(las.classification)
    xyz = np.stack([las.x, las.y, las.z], axis=1)
    return xyz, classes

xyzs, classes = Parallel(n_jobs=-1)(delayed(_read_las)(file) for file in tqdm(filelist))
